In [1]:
_type(::TT) where TT<:Type{T} where T = (TT,T)
_type(obj) = obj |> typeof |> _type

_type (generic function with 2 methods)

In [2]:
_type(Tuple{String,:a})

(DataType, Tuple{String,:a})

In [3]:
_type((:a,:b, 2))

(DataType, Tuple{Symbol,Symbol,Int64})

In [4]:
_type(:a)

(DataType, Symbol)

In [5]:
_type("a")

(DataType, String)

In [6]:
const Maybe{T} = Union{T,Nothing}
@generated function fromjson(::Type{T}, json::Dict) where T
    fromjson_impl(T, json)
end
fromjson(::Type{Maybe{T}}, json::Dict) where T = fromjson(T, json)

fromjson(::Type{T}, ::Nothing) where T = nothing
fromjson(::Type{I}, ::Nothing) where {I<:Integer} = 0
fromjson(::Type{<:AbstractString}, ::Nothing) = ""
fromjson(::Type{VersionNumber}, ::Nothing) = typemin(VersionNumber)
fromjson(::Type{Vector{T}}, ::Nothing) where T = T[]

fromjson(::Type{<:Any}, json) = json
fromjson(::Type{I}, json) where {I<:Integer} = I(json)
fromjson(::Type{<:AbstractString}, json) = string(json)
fromjson(::Type{VersionNumber}, json) = VersionNumber(string(json))
function fromjson(::Type{Vector{T}}, json) where T
    values = T[]
    for item in json
        push!(values, fromjson(T, item))
    end
    values
end

struct Parser{T}
    key::AbstractString
end
Parser(::Type{T}, key) where T = Parser{T}(key)

#= TODO:
For each Type{T}, we can generate a methods that will recursively generate
the expression for parsing all of the fields so that there aren't any
extra steps at runtime. Right now, the generated function isn't recursively
processing the field types
=#
@inline parse(p::Parser{T}, json::Dict) where T = fromjson(T, json[p.key])
@inline parse(p::Parser{Maybe{T}}, json::Dict) where T = begin
    if haskey(json, p.key)
        return fromjson(T, json[p.key])
    end
    nothing
end

function fromjson_impl(::Type{T}, json::Type{<:Dict}) where T
    names = fieldnames(T)
    types = T.types

    ex = Expr(:tuple)
    for i in eachindex(types)
        fname = names[i]
        ftype = types[i]

        key = :( $(string(fname)) )
        parser = :( $(Parser(ftype, key)) )
        push!(ex.args, :( parse($parser, json) ))
    end
    :( $(T)($(ex)...) )
end


fromjson_impl (generic function with 1 method)

In [7]:
module Parse

export Parser

abstract type Parser end
struct ValueParser <: Parser end
struct StructParser{T} <: Parser end

const ValueType = Union{AbstractString,Number,Bool}
Parser(::Type{T}) where T<:ValueType = ValueParser()
Parser(::Type{T}) where T = StructParser{T}()

end

Main.Parse

In [8]:
import Main.Parse

const Parse = Main.Parse
parsers = (
    Parse.Parser(String),
    Parse.Parser(VersionNumber),
    Parse.Parser(Int),
    Parse.Parser(Parse.ValueParser),
    Parse.Parser(Parse.StructParser)
)

(Main.Parse.ValueParser(), Main.Parse.StructParser{VersionNumber}(), Main.Parse.ValueParser(), Main.Parse.StructParser{Main.Parse.ValueParser}(), Main.Parse.StructParser{Main.Parse.StructParser}())

In [37]:
_name(T) = nothing
_name(T::DataType) = T.name
_name(T::UnionAll) = _name(T.body)
_name(T::Union) = string("Union{", join(_names(T.a,T.b), ", "), "}")
_names(args...) = map(_name, map(typeof, args))

structinfo(T,i) = (fieldoffset(T,i), fieldname(T,i), fieldtype(T,i))
structinfo(T) = [structinfo(T,i) for i = 1:fieldcount(T)];
function _t2(dt::TT) where TT<:Type{T} where T
#     dump(TT)
    n = _name(T)
    n !== nothing ? show(n) : nothing
    print("::")
    show(TT.name)
    println()
    for i = 1:fieldcount(TT)
        info = structinfo(TT,i)
        offset,name,typ = info
        print("  ", name, "::", typ, " = ")
        val = isdefined(dt, i) ? getfield(dt, i) : :undefined
        show(val)
        println()
#         show(getproperty(dt, n)); println()
    end
    println()
#     dump(T)
end

struct Getter{T,R}
    index::Int
end
function (g::Getter{T,R})(obj::T)::Maybe{R} where {T,R}
    # TODO: :undefined or nothing ?
    i = g.index
    isdefined(obj, i) ? getfield(obj, i) : nothing
end

struct FieldInfo{T,R}
    offset::UInt
    name::Symbol
    get::Getter{T,R}
end
function FieldInfo(offset::UInt,name::Symbol, getter::Getter{T,R}) where {T,R}
    return FieldInfo{T,R}(offset,name,getter)
end
function FieldInfo(::Type{T},i) where T
    offset,fname,ftype = structinfo(T, i)
    getter = Getter{T,ftype}(i)
    return FieldInfo(offset, fname, getter)
end

struct StructInfo{T,N}
    fields::NTuple{N, FieldInfo{T}}

    function StructInfo{T,N}() where {T,N}
        fields = ntuple(i -> FieldInfo(T,i), N)
        new{T,N}(fields)
    end
end
StructInfo(::Type{T}) where T =  StructInfo{T,fieldcount(T)}()
StructInfo(obj) = StructInfo(typeof(obj))

struct StructInfo2{T, NT <: NamedTuple}
    fields::NT
end
function StructInfo2(::Type{T}) where T 
    N = fieldcount(T)
    finfos = ntuple(i -> FieldInfo(T, i), N)
    names = ntuple(i -> finfos[i].name, N)
    fields = NamedTuple{names}(finfos)
    return StructInfo2{T,typeof(fields)}(fields)
end

StructInfo2

In [38]:
v = v"1.0"
vinfo = StructInfo(v)
for f in vinfo.fields
    info = (f.offset, f.get(v))
    print(f.name)
    print(" = ")
    show(info)
    println()
end

major = (0x0000000000000000, 0x00000001)
minor = (0x0000000000000004, 0x00000000)
patch = (0x0000000000000008, 0x00000000)
prerelease = (0x0000000000000010, ())
build = (0x0000000000000018, ())


In [39]:
@show @code_typed StructInfo(VersionNumber);
@show @code_typed StructInfo2(VersionNumber);

#= In[39]:1 =# @code_typed(StructInfo(VersionNumber)) = CodeInfo(
1 ─ %1  = $(Expr(:static_parameter, 1))::Const(VersionNumber, false)
│   %2  = π (%1, Type{VersionNumber})
│   %3  = (Base.getfield)(%2, :abstract)::Bool
└──       goto #3 if not %3
2 ─       goto #4
3 ─       nothing::Nothing
4 ┄ %7  = φ (#2 => %3, #3 => false)::Bool
└──       goto #6 if not %7
5 ─ %9  = %new(Core.ArgumentError, "type does not have a definite number of fields")::ArgumentError
│         (Base.throw)(%9)::Union{}
└──       $(Expr(:unreachable))::Union{}
6 ┄ %12 = π (%1, Type{VersionNumber})
│   %13 = (Base.getfield)(%12, :types)::SimpleVector
│   %14 = $(Expr(:gc_preserve_begin, :(%13)))
│   %15 = $(Expr(:foreigncall, :(:jl_value_ptr), Ptr{Nothing}, svec(Any), :(:ccall), 1, :(%13)))::Ptr{Nothing}
│   %16 = (Base.bitcast)(Ptr{Int64}, %15)::Ptr{Int64}
│         (Base.pointerref)(%16, 1, 1)::Int64
│         $(Expr(:gc_preserve_end, :(%14)))
└──       goto #7
7 ─ %20 = invoke Main.FieldInfo(VersionNumber::Typ

In [40]:
@code_typed StructInfo{VersionNumber,5}()

CodeInfo(
1 ─ %1 = invoke Main.FieldInfo(VersionNumber::Type{VersionNumber}, 1::Int64)::FieldInfo{VersionNumber,_1} where _1
│   %2 = invoke Main.FieldInfo(VersionNumber::Type{VersionNumber}, 2::Int64)::FieldInfo{VersionNumber,_1} where _1
│   %3 = invoke Main.FieldInfo(VersionNumber::Type{VersionNumber}, 3::Int64)::FieldInfo{VersionNumber,_1} where _1
│   %4 = invoke Main.FieldInfo(VersionNumber::Type{VersionNumber}, 4::Int64)::FieldInfo{VersionNumber,_1} where _1
│   %5 = invoke Main.FieldInfo(VersionNumber::Type{VersionNumber}, 5::Int64)::FieldInfo{VersionNumber,_1} where _1
│   %6 = (Core.tuple)(%1, %2, %3, %4, %5)::NTuple{5,FieldInfo{VersionNumber,_1} where _1}
│   %7 = %new(StructInfo{VersionNumber,5}, %6)::StructInfo{VersionNumber,5}
└──      return %7
) => StructInfo{VersionNumber,5}

In [48]:
# dump(StructInfo(VersionNumber))
info2 = StructInfo2(VersionNumber)
fields = info2.fields
dump(fields)

fieldsT = typeof(fields)
show(fieldsT); println()

for (i,p) in enumerate(fieldsT.parameters)
    print(i, " = ")    
    dump(p)
end


NamedTuple{(:major, :minor, :patch, :prerelease, :build),Tuple{FieldInfo{VersionNumber,UInt32},FieldInfo{VersionNumber,UInt32},FieldInfo{VersionNumber,UInt32},FieldInfo{VersionNumber,Tuple{Vararg{Union{UInt64, String},N} where N}},FieldInfo{VersionNumber,Tuple{Vararg{Union{UInt64, String},N} where N}}}}
  major: FieldInfo{VersionNumber,UInt32}
    offset: UInt64 0x0000000000000000
    name: Symbol major
    get: Getter{VersionNumber,UInt32}
      index: Int64 1
  minor: FieldInfo{VersionNumber,UInt32}
    offset: UInt64 0x0000000000000004
    name: Symbol minor
    get: Getter{VersionNumber,UInt32}
      index: Int64 2
  patch: FieldInfo{VersionNumber,UInt32}
    offset: UInt64 0x0000000000000008
    name: Symbol patch
    get: Getter{VersionNumber,UInt32}
      index: Int64 3
  prerelease: FieldInfo{VersionNumber,Tuple{Vararg{Union{UInt64, String},N} where N}}
    offset: UInt64 0x0000000000000010
    name: Symbol prerelease
    get: Getter{VersionNumber,Tuple{Vararg{Union{UInt64, Str

In [53]:
function Base.NamedTuple(names::NTuple{N,Symbol}, args::T) where {N, T <: Tuple}
    NamedTuple{names,T}(args)
end
function Base.NamedTuple(fields::Vararg{FieldInfo{T},N}) where {T,N}
    names = ntuple(i -> fields[i].name, N)
    return NamedTuple(names, fields)
end
t = NamedTuple((:a,:b), (2,3))
dump(t)
t2 = NamedTuple(info2.fields...)
dump(t2)

NamedTuple{(:a, :b),Tuple{Int64,Int64}}
  a: Int64 2
  b: Int64 3
NamedTuple{(:major, :minor, :patch, :prerelease, :build),Tuple{FieldInfo{VersionNumber,UInt32},FieldInfo{VersionNumber,UInt32},FieldInfo{VersionNumber,UInt32},FieldInfo{VersionNumber,Tuple{Vararg{Union{UInt64, String},N} where N}},FieldInfo{VersionNumber,Tuple{Vararg{Union{UInt64, String},N} where N}}}}
  major: FieldInfo{VersionNumber,UInt32}
    offset: UInt64 0x0000000000000000
    name: Symbol major
    get: Getter{VersionNumber,UInt32}
      index: Int64 1
  minor: FieldInfo{VersionNumber,UInt32}
    offset: UInt64 0x0000000000000004
    name: Symbol minor
    get: Getter{VersionNumber,UInt32}
      index: Int64 2
  patch: FieldInfo{VersionNumber,UInt32}
    offset: UInt64 0x0000000000000008
    name: Symbol patch
    get: Getter{VersionNumber,UInt32}
      index: Int64 3
  prerelease: FieldInfo{VersionNumber,Tuple{Vararg{Union{UInt64, String},N} where N}}
    offset: UInt64 0x0000000000000010
    name: Symbol prere

In [23]:
_t2(Any)
_t2(Core.TypeName)
_t2(DataType)
_t2(AbstractString)
_t2(String)
_t2(SubString)
_t2(VersionNumber)
_t2(Tuple{String,:a})
_t2(Tuple)
_t2(Vector{String})
_t2(Vector)
_t2(Type{String})
_t2(Type)
_t2(UnionAll)
_t2(TypeVar)
_t2(Symbol)
_t2(Union)
_t2(Union{String,Int})

Any::DataType
  name::Core.TypeName = Any
  super::DataType = Any
  parameters::Core.SimpleVector = svec()
  types::Core.SimpleVector = svec()
  names::Core.SimpleVector = :undefined
  instance::Any = :undefined
  layout::Ptr{Nothing} = Ptr{Nothing} @0x0000000000000000
  size::Int32 = 0
  ninitialized::Int32 = 0
  uid::Int32 = 0
  abstract::Bool = true
  mutable::Bool = false
  hasfreetypevars::Bool = false
  isconcretetype::Bool = false
  isdispatchtuple::Bool = false
  isbitstype::Bool = false
  zeroinit::Bool = false
  isinlinealloc::Bool = false
  llvm::StructType::Ptr{Nothing} = Ptr{Nothing} @0x0000000000000000
  llvm::DIType::Ptr{Nothing} = Ptr{Nothing} @0x0000000000000000

Core.TypeName::DataType
  name::Core.TypeName = Core.TypeName
  super::DataType = Any
  parameters::Core.SimpleVector = svec()
  types::Core.SimpleVector = svec(Symbol, Module, SimpleVector, Type, SimpleVector, SimpleVector, Int64, Any)
  names::Core.SimpleVector = :undefined
  instance::Any = :undefined
  lay